# Model Evaluation

In this notebook, we will evaluate the performance of our Kriging model for `NYS Oil & Gas Wells` dataset. We will assess the model’s accuracy and reliability using a range of spatial evaluation metrics. Our goal is to determine the most effective model for this task.

By the end of this notebook, we will identify the strongest model for predicting where to drill for energy, providing valuable insights for further analysis and real-world application.